In [25]:
import pandas as pd
import json

def read_json_file(file_path):
    data_frames = []
    with open(file_path) as f:
        for line in f:
            json_obj = json.loads(line)
            df = pd.json_normalize(json_obj)
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)


df1 = read_json_file('accord.json')
df2 = read_json_file('altima.json')
df3 = read_json_file('camry.json')
df4 = read_json_file('f150.json')

df = pd.concat([df1,df2,df3,df4])

print(df)

                                                   url         id  price  \
0    http://www.autotrader.com//cars-for-sale/vehic...  667544495  33887   
1    http://www.autotrader.com//cars-for-sale/vehic...  673767724  22900   
2    http://www.autotrader.com//cars-for-sale/vehic...  672968554  26386   
3    http://www.autotrader.com//cars-for-sale/vehic...  672335349  28575   
4    http://www.autotrader.com//cars-for-sale/vehic...  671578140  31391   
..                                                 ...        ...    ...   
995  http://www.autotrader.com//cars-for-sale/vehic...  673630388  46380   
996  http://www.autotrader.com//cars-for-sale/vehic...  673550051  46380   
997  http://www.autotrader.com//cars-for-sale/vehic...  673630389  46380   
998  http://www.autotrader.com//cars-for-sale/vehic...  673743503  50730   
999  http://www.autotrader.com//cars-for-sale/vehic...  673630396  50730   

                   vin odometer     trim  year age   make   model saletype  \
0    1HGC

In [26]:
#drop rows with missing data
df = df.loc[~(df == 'Missing').any(axis=1)]
df = df.dropna()
#drop rows where the price is '0'
df = df[df['price'] != 0]
#drop the new listings that snuck in
df = df[df['saletype'] != 'New']


print(df)

                                                   url         id  price  \
0    http://www.autotrader.com//cars-for-sale/vehic...  667544495  33887   
1    http://www.autotrader.com//cars-for-sale/vehic...  673767724  22900   
2    http://www.autotrader.com//cars-for-sale/vehic...  672968554  26386   
3    http://www.autotrader.com//cars-for-sale/vehic...  672335349  28575   
4    http://www.autotrader.com//cars-for-sale/vehic...  671578140  31391   
..                                                 ...        ...    ...   
974  http://www.autotrader.com//cars-for-sale/vehic...  669518397  18990   
975  http://www.autotrader.com//cars-for-sale/vehic...  673009955  48945   
976  http://www.autotrader.com//cars-for-sale/vehic...  671216983  30000   
979  http://www.autotrader.com//cars-for-sale/vehic...  672091197  20995   
992  http://www.autotrader.com//cars-for-sale/vehic...  673551474  42500   

                   vin odometer                                       trim  \
0    1HGC

In [27]:
#Break down features column

# apply function to create new columns
def create_columns(row):
    unique_values = pd.Series(row['features']).explode().unique()
    for value in unique_values:
        row[value] = value in row['features']
    return row

df = df.apply(create_columns, axis=1)

# drop the original column
df.drop('features', axis=1, inplace=True)

In [28]:
df

,Driver,#1 Price in Market,(1) 12V aux pwr outlet,(1) front passenger seat integrated child seat tether,(2) 12V aux pwr outlets,(2) 12V auxiliary pwr outlets,(2) 12V pwr outlets,(2) front tow hooks,(2) instrument panel cupholders,(2) rear cupholders,...,push-button ignition,saletype,taillights and running lights),trim,url,v8,vin,we are excited about this one...,wider and longer than the outgoing model\\u003cbr />Redesigned interior with improved touchscreen system\\u003cbr />Part of the 10th Accord generation introduced for 2018\\u003cbr />Standard features on the base LX model include 17-inch alloy wheels,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,Touring,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1HGCV3F91LA015984,NaN,NaN,2020
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,Sport,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1HGCV1F36JA039324,NaN,NaN,2018
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,Sport,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1HGCV1F38KA100917,NaN,NaN,2019
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,Sport,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1HGCV1F32LA060903,NaN,NaN,2020
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,EX-L,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1HGCV1F5XMA079539,NaN,NaN,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,FX4 w/ Luxury Equipment Group,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1FTFW1ET7DKE82283,NaN,NaN,2013
975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,XLT w/ XLT Sport Appearance Package,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1FTFW1E87MFB61624,NaN,NaN,2021
976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,King Ranch w/ Equipment Group 601A Luxury,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1FTEW1EG2JFB69564,NaN,NaN,2018
979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Used,NaN,FX4 w/ Luxury Equipment Group,http://www.autotrader.com//cars-for-sale/vehic...,NaN,1FTFW1EF0DKG22846,NaN,NaN,2013


In [34]:
# find columns with less than 10 True values
to_drop = [col for col in df.columns if (df[col] == True).sum() <= 10]

print(to_drop)

[' Driver', '(2) instrument panel cupholders', '(4) full-size doors', '(4) pickup box tie-down hooks', '* BACK UP CAMERA \\\\u003cBR>', '* BACK UP SENSORS *', '* BLUETOOTH \\\\u003cBR>', '* Best Low', '* CLEAN CARFAX \\\\u003cBR>', '* KEYLESS START \\\\u003cBR>', '* MULTI-POINT DEALER INSPECTION * \\\\u003cBR>', '* NAVIGATION *', '* Pre-qualified for 3 year / 45', '* RUNNING BOARDS *', '* TOWING PKG *', '* VEHICLE DETAILED * \\\\u003cBR>\\\\u003cBR>', '** 4WD/4x4', '** 4X4 - NEVER WORRY ABOUT THE WEATHER **', '** AUTOMATIC DUAL TEMPERATURE CONTROL **', '** Adaptive Cruise Control', '** Adaptive Suspension', '** Alloy Wheels', '** Apple CarPlay/Andriod Auto', '** Automatic Emergency Braking', '** BEDLINER **', '** BLIS **', '** BLUETOOTH HANDS FREE CALLING **', '** Blind Spot Assist', '** Blind Spot Monitoring', '** CARFAX ONE OWNER **', '** CRUISE CONTROL **', '** FORD CO PILOT 360 **', '** Front BUCKET SEATS **', '** HEATED SEATS **', '** Heated Steering Wheel', '** Homelink', '** INT

In [35]:
# drop the columns
df.drop(to_drop, axis=1, inplace=True)

In [38]:
df

,* AUTOMATIC *,* BACK UP CAMERA *,* BLUETOOTH *,* HONDA CERTIFIED *,* LEATHER INTERIOR *,** 4 X 4 **,** ALLOY WHEELS **,** Alloy Wheels / Premium Wheels,** Apple CarPlay / Android Auto,** BACK UP CAMERA **,...,Woodgrain Interior Trim,XL Power Equipment Group,XL Sport Appearance Package,XLT Chrome Appearance Package,XLT Power Equipment Group,XLT Sport Appearance Package,Zone Lighting,[B10,age,iPod/MP3 Input
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN
975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,True,NaN,NaN,2,NaN
976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN


In [40]:
df.to_csv('cleaned.csv', index=False)